## Deliverable 3. Create a Travel Itinerary Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Lat,Lng,Hotel Name
0,1,Praya,CV,77.54,73,20,12.66,few clouds,14.9215,-23.5087,LT Aparthotel
1,2,Springbok,ZA,65.98,14,32,6.15,scattered clouds,-29.6643,17.8865,Undulata Country Lodge
2,3,Hilo,US,75.79,94,90,3.44,mist,19.7297,-155.0900,Hilo Hawaiian Hotel
3,7,Atuona,PF,77.99,74,9,17.31,clear sky,-9.8000,-139.0333,Villa Enata
4,9,Vao,NC,71.31,86,91,9.98,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny


In [22]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
city_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

,Lat,Lng
0,14.9215,-23.5087
1,-29.6643,17.8865
2,19.7297,-155.0900
3,-9.8000,-139.0333
4,-22.6667,167.4833
...,...,...
261,40.4272,71.7189
262,31.6682,20.2521
263,17.9833,-102.2333
264,3.3803,-59.7968


In [7]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]== "Coahuayana"]
vacation_end = vacation_df.loc[vacation_df["City"]== "Coahuayana"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]== "Tecalitlan"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]== "Iramuco"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]== "La Orilla"]

vacation_start

,City_ID,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Lat,Lng,Hotel Name
75,205,Coahuayana,MX,76.64,94,95,3.42,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos


In [40]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]].to_numpy()
end = vacation_end[["Lat", "Lng"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()

In [41]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
direction_from_atob = gmaps.directions_layer(
        start[0], end[0], waypoints=[stop1[0], stop2[0], stop3[0]],
        travel_mode='DRIVING')
fig.add_layer(direction_from_atob)
fig


Figure(layout=FigureLayout(height='420px'))

In [42]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Lat,Lng,Hotel Name
0,205,Coahuayana,MX,76.64,94,95,3.42,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
1,369,Tecalitlan,MX,63.55,98,93,1.30,overcast clouds,19.4667,-103.3000,Hotel Tierra Bella
2,378,Iramuco,MX,62.71,88,40,0.00,scattered clouds,19.9500,-100.9167,La Hacienda Centro Cultural y de Entretenimiento
3,691,La Orilla,MX,77.25,92,79,4.12,broken clouds,17.9833,-102.2333,Baymont by Wyndham Lazaro Cardenas


In [45]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
city_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))